In [1]:
# import the libraries
import torch
import numpy as np
import networkx as nx
import matplotlib.pyplot as plt
from sklearn.manifold import TSNE

# import the functions
from numpy import dot
import torch.nn as nn
import torch_geometric.utils as torch_utils
from sklearn import metrics
from numpy.linalg import norm
import torch.nn.functional as F
from sklearn.metrics import roc_auc_score
from torch_geometric.utils import mask as Mask
from torch_geometric.utils import to_networkx
from torch_geometric.nn.models import Node2Vec
from sklearn.metrics import precision_recall_curve
from sklearn.metrics import average_precision_score
from sklearn.model_selection import train_test_split
from sklearn.manifold import TSNE as tsne

import subprocess
import requests
import os
import pandas as pd
import numpy as np
import scipy as sp
import networkx as nx
import matplotlib.pyplot as plt
from matplotlib.collections import LineCollection
import sys

In [2]:
data = pd.read_csv('/Users/mac/Documents/FILES/TUDelft/Q4/Machine Learning for Graph Data/project/aggregated.csv')

/Users/mac/opt/anaconda3/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3165: DtypeWarning: Columns (2,3,4,95,96,204,205,206,207,208,209,210,211,212,213,234,235,684,685) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


In [3]:
data = data[data['aggregation_level'] == 2]

In [4]:
data = data[data['country_code'] == 'US']

In [5]:
data

,location_key,date,place_id,wikidata_id,datacommons_id,country_code,country_name,iso_3166_1_alpha_2,iso_3166_1_alpha_3,aggregation_level,...,new_persons_fully_vaccinated_janssen,cumulative_persons_fully_vaccinated_janssen,new_persons_vaccinated_sinovac,total_persons_vaccinated_sinovac,new_vaccine_doses_administered_pfizer,cumulative_vaccine_doses_administered_pfizer,new_vaccine_doses_administered_moderna,cumulative_vaccine_doses_administered_moderna,new_vaccine_doses_administered_janssen,cumulative_vaccine_doses_administered_janssen
19382969,US_AK_02013,2020-01-01,ChIJ7a-UgVuW_lYRjTetzbIWL48,Q504371,geoId/02013,US,United States of America,US,USA,2,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
19382970,US_AK_02013,2020-01-02,ChIJ7a-UgVuW_lYRjTetzbIWL48,Q504371,geoId/02013,US,United States of America,US,USA,2,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
19382971,US_AK_02013,2020-01-03,ChIJ7a-UgVuW_lYRjTetzbIWL48,Q504371,geoId/02013,US,United States of America,US,USA,2,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
19382972,US_AK_02013,2020-01-04,ChIJ7a-UgVuW_lYRjTetzbIWL48,Q504371,geoId/02013,US,United States of America,US,USA,2,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
19382973,US_AK_02013,2020-01-05,ChIJ7a-UgVuW_lYRjTetzbIWL48,Q504371,geoId/02013,US,United States of America,US,USA,2,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
22636417,US_WY_56045,2022-09-13,ChIJd4Rqhed3YocR7ubT5-HgoJg,Q115413,geoId/56045,US,United States of America,US,USA,2,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
22636418,US_WY_56045,2022-09-14,ChIJd4Rqhed3YocR7ubT5-HgoJg,Q115413,geoId/56045,US,United States of America,US,USA,2,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
22636419,US_WY_56045,2022-09-15,ChIJd4Rqhed3YocR7ubT5-HgoJg,Q115413,geoId/56045,US,United States of America,US,USA,2,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
22636420,US_WY_56045,2022-09-16,ChIJd4Rqhed3YocR7ubT5-HgoJg,Q115413,geoId/56045,US,United States of America,US,USA,2,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [10]:
features=['location_key', 'date', 'population', 'population_density', 'new_recovered', 'new_deceased', 'new_confirmed', 
          'latitude', 'longitude', 'smoking_prevalence', 'diabetes_prevalence', 'infant_mortality_rate', 'nurses_per_1000',
          'physicians_per_1000', 'health_expenditure_usd', 'out_of_pocket_health_expenditure_usd', 'life_expectancy',
          'adult_male_mortality_rate', 'adult_female_mortality_rate', 'pollution_mortality_rate', 'comorbidity_mortality_rate',
          'mobility_retail_and_recreation', 'mobility_grocery_and_pharmacy', 'mobility_parks', 'mobility_transit_stations',
          'mobility_workplaces', 'mobility_residential', 'search_trends_abdominal_obesity', 'search_trends_abdominal_pain',
          'search_trends_acne', 'search_trends_actinic_keratosis', 'search_trends_acute_bronchitis', 'search_trends_adrenal_crisis',
          'search_trends_ageusia', 'search_trends_alcoholism', 'search_trends_allergic_conjunctivitis', 'search_trends_allergy',
          'search_trends_amblyopia', 'search_trends_amenorrhea', 'search_trends_amnesia', 'search_trends_anal_fissure',
          'search_trends_anaphylaxis', 'search_trends_anemia', 'search_trends_angina_pectoris', 'search_trends_angioedema',
          'search_trends_angular_cheilitis', 'search_trends_anosmia', 'search_trends_anxiety', 'search_trends_aphasia',
          'search_trends_aphonia', 'search_trends_apnea', 'search_trends_arthralgia', 'search_trends_arthritis',
          'search_trends_ascites', 'search_trends_asperger_syndrome', 'search_trends_asphyxia', 'search_trends_asthma',
          'search_trends_astigmatism', 'search_trends_ataxia', 'search_trends_atheroma', 
          'search_trends_attention_deficit_hyperactivity_disorder', 'search_trends_auditory_hallucination',
          'search_trends_autoimmune_disease', 'search_trends_avoidant_personality_disorder',
          'search_trends_back_pain', 'search_trends_bacterial_vaginosis',
          'search_trends_balance_disorder',
          'search_trends_beaus_lines',
          'search_trends_bells_palsy',
          'search_trends_biliary_colic',
          'search_trends_binge_eating',
          'search_trends_bleeding',
          'search_trends_bleeding_on_probing',
          'search_trends_blepharospasm',
          'search_trends_bloating',
          'search_trends_blood_in_stool',
          'search_trends_blurred_vision',
          'search_trends_blushing',
          'search_trends_boil',
          'search_trends_bone_fracture',
          'search_trends_bone_tumor',
          'search_trends_bowel_obstruction',
          'search_trends_bradycardia',
          'search_trends_braxton_hicks_contractions',
          'search_trends_breakthrough_bleeding',
          'search_trends_breast_pain',
          'search_trends_bronchitis',
          'search_trends_bruise',
          'search_trends_bruxism',
          'search_trends_bunion',
          'search_trends_burn',
          'search_trends_burning_chest_pain',
          'search_trends_burning_mouth_syndrome',
          'search_trends_candidiasis',
          'search_trends_canker_sore',
          'search_trends_cardiac_arrest',
          'search_trends_carpal_tunnel_syndrome',
          'search_trends_cataplexy',
          'search_trends_cataract',
          'search_trends_chancre',
          'search_trends_cheilitis',
          'search_trends_chest_pain',
          'search_trends_chills',
          'search_trends_chorea',
          'search_trends_chronic_pain',
          'search_trends_cirrhosis',
          'search_trends_cleft_lip_and_cleft_palate',
          'search_trends_clouding_of_consciousness',
          'search_trends_cluster_headache',
          'search_trends_colitis',
          'search_trends_coma',
          'search_trends_common_cold',
          'search_trends_compulsive_behavior',
          'search_trends_compulsive_hoarding',
          'search_trends_confusion',
          'search_trends_congenital_heart_defect',
          'search_trends_conjunctivitis',
          'search_trends_constipation',
          'search_trends_convulsion',
          'search_trends_cough',
          'search_trends_crackles',
          'search_trends_cramp',
          'search_trends_crepitus',
          'search_trends_croup',
          'search_trends_cyanosis',
          'search_trends_dandruff',
          'search_trends_delayed_onset_muscle_soreness',
          'search_trends_dementia',
          'search_trends_dentin_hypersensitivity',
          'search_trends_depersonalization',
          'search_trends_depression',
          'search_trends_dermatitis',
          'search_trends_desquamation',
          'search_trends_developmental_disability',
          'search_trends_diabetes',
          'search_trends_diabetic_ketoacidosis',
          'search_trends_diarrhea',
          'search_trends_dizziness',
          'search_trends_dry_eye_syndrome',
          'search_trends_dysautonomia',
          'search_trends_dysgeusia',
          'search_trends_dysmenorrhea',
          'search_trends_dyspareunia',
          'search_trends_dysphagia',
          'search_trends_dysphoria',
          'search_trends_dystonia',
          'search_trends_dysuria',
          'search_trends_ear_pain',
          'search_trends_eczema',
          'search_trends_edema',
          'search_trends_encephalitis',
          'search_trends_encephalopathy',
          'search_trends_epidermoid_cyst',
          'search_trends_epilepsy',
          'search_trends_epiphora',
          'search_trends_erectile_dysfunction',
          'search_trends_erythema',
          'search_trends_erythema_chronicum_migrans',
          'search_trends_esophagitis',
          'search_trends_excessive_daytime_sleepiness',
          'search_trends_eye_pain',
          'search_trends_eye_strain',
          'search_trends_facial_nerve_paralysis',
          'search_trends_facial_swelling',
          'search_trends_fasciculation',
          'search_trends_fatigue',
          'search_trends_fatty_liver_disease',
          'search_trends_fecal_incontinence',
          'search_trends_fever',
          'search_trends_fibrillation',
          'search_trends_fibrocystic_breast_changes',
          'search_trends_fibromyalgia',
          'search_trends_flatulence',
          'search_trends_floater',
          'search_trends_focal_seizure',
          'search_trends_folate_deficiency',
          'search_trends_food_craving',
          'search_trends_food_intolerance',
          'search_trends_frequent_urination',
          'search_trends_gastroesophageal_reflux_disease',
          'search_trends_gastroparesis',
          'search_trends_generalized_anxiety_disorder',
          'search_trends_genital_wart',
          'search_trends_gingival_recession',
          'search_trends_gingivitis',
          'search_trends_globus_pharyngis',
          'search_trends_goitre',
          'search_trends_gout',
          'search_trends_grandiosity',
          'search_trends_granuloma',
          'search_trends_guilt',
          'search_trends_hair_loss',
          'search_trends_halitosis',
          'search_trends_hay_fever',
          'search_trends_headache',
          'search_trends_heart_arrhythmia',
          'search_trends_heart_murmur',
          'search_trends_heartburn',
          'search_trends_hematochezia',
          'search_trends_hematoma',
          'search_trends_hematuria',
          'search_trends_hemolysis',
          'search_trends_hemoptysis',
          'search_trends_hemorrhoids',
          'search_trends_hepatic_encephalopathy',
          'search_trends_hepatitis',
          'search_trends_hepatotoxicity',
          'search_trends_hiccup',
          'search_trends_hip_pain',
          'search_trends_hives',
          'search_trends_hot_flash',
          'search_trends_hydrocephalus',
          'search_trends_hypercalcaemia',
          'search_trends_hypercapnia',
          'search_trends_hypercholesterolemia',
          'search_trends_hyperemesis_gravidarum',
          'search_trends_hyperglycemia',
          'search_trends_hyperhidrosis',
          'search_trends_hyperkalemia',
          'search_trends_hyperlipidemia',
          'search_trends_hypermobility',
          'search_trends_hyperpigmentation',
          'search_trends_hypersomnia',
          'search_trends_hypertension',
          'search_trends_hyperthermia',
          'search_trends_hyperthyroidism',
          'search_trends_hypertriglyceridemia',
          'search_trends_hypertrophy',
          'search_trends_hyperventilation',
          'search_trends_hypocalcaemia',
          'search_trends_hypochondriasis',
          'search_trends_hypoglycemia',
          'search_trends_hypogonadism',
          'search_trends_hypokalemia',
          'search_trends_hypomania',
          'search_trends_hyponatremia',
          'search_trends_hypotension',
          'search_trends_hypothyroidism',
          'search_trends_hypoxemia',
          'search_trends_hypoxia',
          'search_trends_impetigo',
          'search_trends_implantation_bleeding',
          'search_trends_impulsivity',
          'search_trends_indigestion',
          'search_trends_infection',
          'search_trends_inflammation',
          'search_trends_inflammatory_bowel_disease',
          'search_trends_ingrown_hair',
          'search_trends_insomnia',
          'search_trends_insulin_resistance',
          'search_trends_intermenstrual_bleeding',
          'search_trends_intracranial_pressure',
          'search_trends_iron_deficiency',
          'search_trends_irregular_menstruation',
          'search_trends_itch',
          'search_trends_jaundice',
          'search_trends_kidney_failure',
          'search_trends_kidney_stone',
          'search_trends_knee_pain',
          'search_trends_kyphosis',
          'search_trends_lactose_intolerance',
          'search_trends_laryngitis',
          'search_trends_leg_cramps',
          'search_trends_lesion',
          'search_trends_leukorrhea',
          'search_trends_lightheadedness',
          'search_trends_low_back_pain',
          'search_trends_low_grade_fever',
          'search_trends_lymphedema',
          'search_trends_major_depressive_disorder',
          'search_trends_malabsorption',
          'search_trends_male_infertility',
          'search_trends_manic_disorder',
          'search_trends_melasma',
          'search_trends_melena',
          'search_trends_meningitis',
          'search_trends_menorrhagia',
          'search_trends_middle_back_pain',
          'search_trends_migraine',
          'search_trends_milium',
          'search_trends_mitral_insufficiency',
          'search_trends_mood_disorder',
          'search_trends_mood_swing',
          'search_trends_morning_sickness',
          'search_trends_motion_sickness',
          'search_trends_mouth_ulcer',
          'search_trends_muscle_atrophy',
          'search_trends_muscle_weakness',
          'search_trends_myalgia',
          'search_trends_mydriasis',
          'search_trends_myocardial_infarction',
          'search_trends_myoclonus',
          'search_trends_nasal_congestion',
          'search_trends_nasal_polyp',
          'search_trends_nausea',
          'search_trends_neck_mass',
          'search_trends_neck_pain',
          'search_trends_neonatal_jaundice',
          'search_trends_nerve_injury',
          'search_trends_neuralgia',
          'search_trends_neutropenia',
          'search_trends_night_sweats',
          'search_trends_night_terror',
          'search_trends_nocturnal_enuresis',
          'search_trends_nodule',
          'search_trends_nosebleed',
          'search_trends_nystagmus',
          'search_trends_obesity',
          'search_trends_onychorrhexis',
          'search_trends_oral_candidiasis',
          'search_trends_orthostatic_hypotension',
          'search_trends_osteopenia',
          'search_trends_osteophyte',
          'search_trends_osteoporosis',
          'search_trends_otitis',
          'search_trends_otitis_externa',
          'search_trends_otitis_media',
          'search_trends_pain',
          'search_trends_palpitations',
          'search_trends_pancreatitis',
          'search_trends_panic_attack',
          'search_trends_papule',
          'search_trends_paranoia',
          'search_trends_paresthesia',
          'search_trends_pelvic_inflammatory_disease',
          'search_trends_pericarditis',
          'search_trends_periodontal_disease',
          'search_trends_periorbital_puffiness',
          'search_trends_peripheral_neuropathy',
          'search_trends_perspiration',
          'search_trends_petechia',
          'search_trends_phlegm',
          'search_trends_photodermatitis',
          'search_trends_photophobia',
          'search_trends_photopsia',
          'search_trends_pleural_effusion',
          'search_trends_pleurisy',
          'search_trends_pneumonia',
          'search_trends_podalgia',
          'search_trends_polycythemia',
          'search_trends_polydipsia',
          'search_trends_polyneuropathy',
          'search_trends_polyuria',
          'search_trends_poor_posture',
          'search_trends_post_nasal_drip',
          'search_trends_postural_orthostatic_tachycardia_syndrome',
          'search_trends_prediabetes',
          'search_trends_proteinuria',
          'search_trends_pruritus_ani',
          'search_trends_psychosis',
          'search_trends_ptosis',
          'search_trends_pulmonary_edema',
          'search_trends_pulmonary_hypertension',
          'search_trends_purpura',
          'search_trends_pus',
          'search_trends_pyelonephritis',
          'search_trends_radiculopathy',
          'search_trends_rectal_pain',
          'search_trends_rectal_prolapse',
          'search_trends_red_eye',
          'search_trends_renal_colic',
          'search_trends_restless_legs_syndrome',
          'search_trends_rheum',
          'search_trends_rhinitis',
          'search_trends_rhinorrhea',
          'search_trends_rosacea',
          'search_trends_round_ligament_pain',
          'search_trends_rumination',
          'search_trends_scar',
          'search_trends_sciatica',
          'search_trends_scoliosis',
          'search_trends_seborrheic_dermatitis',
          'search_trends_self_harm',
          'search_trends_sensitivity_to_sound',
          'search_trends_sexual_dysfunction',
          'search_trends_shallow_breathing',
          'search_trends_sharp_pain',
          'search_trends_shivering',
          'search_trends_shortness_of_breath',
          'search_trends_shyness',
          'search_trends_sinusitis',
          'search_trends_skin_condition',
          'search_trends_skin_rash',
          'search_trends_skin_tag',
          'search_trends_skin_ulcer',
          'search_trends_sleep_apnea',
          'search_trends_sleep_deprivation',
          'search_trends_sleep_disorder',
          'search_trends_snoring',
          'search_trends_sore_throat',
          'search_trends_spasticity',
          'search_trends_splenomegaly',
          'search_trends_sputum',
          'search_trends_stomach_rumble',
          'search_trends_strabismus',
          'search_trends_stretch_marks',
          'search_trends_stridor',
          'search_trends_stroke',
          'search_trends_stuttering',
          'search_trends_subdural_hematoma',
          'search_trends_suicidal_ideation',
          'search_trends_swelling',
          'search_trends_swollen_feet',
          'search_trends_swollen_lymph_nodes',
          'search_trends_syncope',
          'search_trends_tachycardia',
          'search_trends_tachypnea',
          'search_trends_telangiectasia',
          'search_trends_tenderness',
          'search_trends_testicular_pain',
          'search_trends_throat_irritation',
          'search_trends_thrombocytopenia',
          'search_trends_thyroid_nodule',
          'search_trends_tic',
          'search_trends_tinnitus',
          'search_trends_tonsillitis',
          'search_trends_toothache',
          'search_trends_tremor',
          'search_trends_trichoptilosis',
          'search_trends_tumor',
          'search_trends_type_2_diabetes',
          'search_trends_unconsciousness',
          'search_trends_underweight',
          'search_trends_upper_respiratory_tract_infection',
          'search_trends_urethritis',
          'search_trends_urinary_incontinence',
          'search_trends_urinary_tract_infection',
          'search_trends_urinary_urgency',
          'search_trends_uterine_contraction',
          'search_trends_vaginal_bleeding',
          'search_trends_vaginal_discharge',
          'search_trends_vaginitis',
          'search_trends_varicose_veins',
          'search_trends_vasculitis',
          'search_trends_ventricular_fibrillation',
          'search_trends_ventricular_tachycardia',
          'search_trends_vertigo',
          'search_trends_viral_pneumonia',
          'search_trends_visual_acuity',
          'search_trends_vomiting',
          'search_trends_wart',
          'search_trends_water_retention',
          'search_trends_weakness',
          'search_trends_weight_gain',
          'search_trends_wheeze',
          'search_trends_xeroderma',
          'search_trends_xerostomia',
          'search_trends_yawn',
          'new_persons_vaccinated',
          'cumulative_persons_vaccinated',
          'stringency_index',
          'average_temperature_celsius',
          'minimum_temperature_celsius',
          'maximum_temperature_celsius',
          'rainfall_mm',
          'snowfall_mm',
          'dew_point',
          'relative_humidity']

In [11]:
data = data.loc[:, features]

In [12]:
data

,location_key,date,population,population_density,new_recovered,new_deceased,new_confirmed,latitude,longitude,smoking_prevalence,...,new_persons_vaccinated,cumulative_persons_vaccinated,stringency_index,average_temperature_celsius,minimum_temperature_celsius,maximum_temperature_celsius,rainfall_mm,snowfall_mm,dew_point,relative_humidity
19382969,US_AK_02013,2020-01-01,3455.0,NaN,NaN,NaN,NaN,55.23,-161.92,NaN,...,NaN,NaN,NaN,-8.583333,-9.590278,-6.819444,0.41275,NaN,-12.077778,78.113561
19382970,US_AK_02013,2020-01-02,3455.0,NaN,NaN,NaN,NaN,55.23,-161.92,NaN,...,NaN,NaN,NaN,-8.854167,-10.770833,-7.701389,0.50800,NaN,-12.811111,75.823558
19382971,US_AK_02013,2020-01-03,3455.0,NaN,NaN,NaN,NaN,55.23,-161.92,NaN,...,NaN,NaN,NaN,-10.368056,-12.486111,-6.770833,0.44450,NaN,-14.577778,75.132879
19382972,US_AK_02013,2020-01-04,3455.0,NaN,NaN,NaN,NaN,55.23,-161.92,NaN,...,NaN,NaN,NaN,-6.944444,-9.993056,-3.937500,0.28575,NaN,-11.711111,73.476473
19382973,US_AK_02013,2020-01-05,3455.0,NaN,NaN,NaN,NaN,55.23,-161.92,NaN,...,NaN,NaN,NaN,-3.791667,-8.291667,1.013889,0.03175,NaN,-7.177778,82.382291
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
22636417,US_WY_56045,2022-09-13,6968.0,NaN,NaN,NaN,NaN,43.84,-104.56,NaN,...,NaN,NaN,NaN,17.683333,6.338889,28.733333,0.00000,NaN,4.055556,39.426422
22636418,US_WY_56045,2022-09-14,6968.0,NaN,NaN,NaN,NaN,43.84,-104.56,NaN,...,NaN,NaN,NaN,20.333333,17.494444,23.766667,0.00000,NaN,4.562500,36.975309
22636419,US_WY_56045,2022-09-15,6968.0,NaN,NaN,NaN,NaN,43.84,-104.56,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
22636420,US_WY_56045,2022-09-16,6968.0,NaN,NaN,NaN,NaN,43.84,-104.56,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [13]:
date = data['date'].unique()
location_key = data['location_key'].unique()

In [16]:
print(date.shape)
print(location_key.shape)

(991,)
(3225,)


In [21]:
features=['population', 'population_density', 'new_recovered', 'new_deceased', 'new_confirmed', 
          'latitude', 'longitude', 'smoking_prevalence', 'diabetes_prevalence', 'infant_mortality_rate', 'nurses_per_1000',
          'physicians_per_1000', 'health_expenditure_usd', 'out_of_pocket_health_expenditure_usd', 'life_expectancy',
          'adult_male_mortality_rate', 'adult_female_mortality_rate', 'pollution_mortality_rate', 'comorbidity_mortality_rate',
          'mobility_retail_and_recreation', 'mobility_grocery_and_pharmacy', 'mobility_parks', 'mobility_transit_stations',
          'mobility_workplaces', 'mobility_residential', 'search_trends_abdominal_obesity', 'search_trends_abdominal_pain',
          'search_trends_acne', 'search_trends_actinic_keratosis', 'search_trends_acute_bronchitis', 'search_trends_adrenal_crisis',
          'search_trends_ageusia', 'search_trends_alcoholism', 'search_trends_allergic_conjunctivitis', 'search_trends_allergy',
          'search_trends_amblyopia', 'search_trends_amenorrhea', 'search_trends_amnesia', 'search_trends_anal_fissure',
          'search_trends_anaphylaxis', 'search_trends_anemia', 'search_trends_angina_pectoris', 'search_trends_angioedema',
          'search_trends_angular_cheilitis', 'search_trends_anosmia', 'search_trends_anxiety', 'search_trends_aphasia',
          'search_trends_aphonia', 'search_trends_apnea', 'search_trends_arthralgia', 'search_trends_arthritis',
          'search_trends_ascites', 'search_trends_asperger_syndrome', 'search_trends_asphyxia', 'search_trends_asthma',
          'search_trends_astigmatism', 'search_trends_ataxia', 'search_trends_atheroma', 
          'search_trends_attention_deficit_hyperactivity_disorder', 'search_trends_auditory_hallucination',
          'search_trends_autoimmune_disease', 'search_trends_avoidant_personality_disorder',
          'search_trends_back_pain', 'search_trends_bacterial_vaginosis',
          'search_trends_balance_disorder',
          'search_trends_beaus_lines',
          'search_trends_bells_palsy',
          'search_trends_biliary_colic',
          'search_trends_binge_eating',
          'search_trends_bleeding',
          'search_trends_bleeding_on_probing',
          'search_trends_blepharospasm',
          'search_trends_bloating',
          'search_trends_blood_in_stool',
          'search_trends_blurred_vision',
          'search_trends_blushing',
          'search_trends_boil',
          'search_trends_bone_fracture',
          'search_trends_bone_tumor',
          'search_trends_bowel_obstruction',
          'search_trends_bradycardia',
          'search_trends_braxton_hicks_contractions',
          'search_trends_breakthrough_bleeding',
          'search_trends_breast_pain',
          'search_trends_bronchitis',
          'search_trends_bruise',
          'search_trends_bruxism',
          'search_trends_bunion',
          'search_trends_burn',
          'search_trends_burning_chest_pain',
          'search_trends_burning_mouth_syndrome',
          'search_trends_candidiasis',
          'search_trends_canker_sore',
          'search_trends_cardiac_arrest',
          'search_trends_carpal_tunnel_syndrome',
          'search_trends_cataplexy',
          'search_trends_cataract',
          'search_trends_chancre',
          'search_trends_cheilitis',
          'search_trends_chest_pain',
          'search_trends_chills',
          'search_trends_chorea',
          'search_trends_chronic_pain',
          'search_trends_cirrhosis',
          'search_trends_cleft_lip_and_cleft_palate',
          'search_trends_clouding_of_consciousness',
          'search_trends_cluster_headache',
          'search_trends_colitis',
          'search_trends_coma',
          'search_trends_common_cold',
          'search_trends_compulsive_behavior',
          'search_trends_compulsive_hoarding',
          'search_trends_confusion',
          'search_trends_congenital_heart_defect',
          'search_trends_conjunctivitis',
          'search_trends_constipation',
          'search_trends_convulsion',
          'search_trends_cough',
          'search_trends_crackles',
          'search_trends_cramp',
          'search_trends_crepitus',
          'search_trends_croup',
          'search_trends_cyanosis',
          'search_trends_dandruff',
          'search_trends_delayed_onset_muscle_soreness',
          'search_trends_dementia',
          'search_trends_dentin_hypersensitivity',
          'search_trends_depersonalization',
          'search_trends_depression',
          'search_trends_dermatitis',
          'search_trends_desquamation',
          'search_trends_developmental_disability',
          'search_trends_diabetes',
          'search_trends_diabetic_ketoacidosis',
          'search_trends_diarrhea',
          'search_trends_dizziness',
          'search_trends_dry_eye_syndrome',
          'search_trends_dysautonomia',
          'search_trends_dysgeusia',
          'search_trends_dysmenorrhea',
          'search_trends_dyspareunia',
          'search_trends_dysphagia',
          'search_trends_dysphoria',
          'search_trends_dystonia',
          'search_trends_dysuria',
          'search_trends_ear_pain',
          'search_trends_eczema',
          'search_trends_edema',
          'search_trends_encephalitis',
          'search_trends_encephalopathy',
          'search_trends_epidermoid_cyst',
          'search_trends_epilepsy',
          'search_trends_epiphora',
          'search_trends_erectile_dysfunction',
          'search_trends_erythema',
          'search_trends_erythema_chronicum_migrans',
          'search_trends_esophagitis',
          'search_trends_excessive_daytime_sleepiness',
          'search_trends_eye_pain',
          'search_trends_eye_strain',
          'search_trends_facial_nerve_paralysis',
          'search_trends_facial_swelling',
          'search_trends_fasciculation',
          'search_trends_fatigue',
          'search_trends_fatty_liver_disease',
          'search_trends_fecal_incontinence',
          'search_trends_fever',
          'search_trends_fibrillation',
          'search_trends_fibrocystic_breast_changes',
          'search_trends_fibromyalgia',
          'search_trends_flatulence',
          'search_trends_floater',
          'search_trends_focal_seizure',
          'search_trends_folate_deficiency',
          'search_trends_food_craving',
          'search_trends_food_intolerance',
          'search_trends_frequent_urination',
          'search_trends_gastroesophageal_reflux_disease',
          'search_trends_gastroparesis',
          'search_trends_generalized_anxiety_disorder',
          'search_trends_genital_wart',
          'search_trends_gingival_recession',
          'search_trends_gingivitis',
          'search_trends_globus_pharyngis',
          'search_trends_goitre',
          'search_trends_gout',
          'search_trends_grandiosity',
          'search_trends_granuloma',
          'search_trends_guilt',
          'search_trends_hair_loss',
          'search_trends_halitosis',
          'search_trends_hay_fever',
          'search_trends_headache',
          'search_trends_heart_arrhythmia',
          'search_trends_heart_murmur',
          'search_trends_heartburn',
          'search_trends_hematochezia',
          'search_trends_hematoma',
          'search_trends_hematuria',
          'search_trends_hemolysis',
          'search_trends_hemoptysis',
          'search_trends_hemorrhoids',
          'search_trends_hepatic_encephalopathy',
          'search_trends_hepatitis',
          'search_trends_hepatotoxicity',
          'search_trends_hiccup',
          'search_trends_hip_pain',
          'search_trends_hives',
          'search_trends_hot_flash',
          'search_trends_hydrocephalus',
          'search_trends_hypercalcaemia',
          'search_trends_hypercapnia',
          'search_trends_hypercholesterolemia',
          'search_trends_hyperemesis_gravidarum',
          'search_trends_hyperglycemia',
          'search_trends_hyperhidrosis',
          'search_trends_hyperkalemia',
          'search_trends_hyperlipidemia',
          'search_trends_hypermobility',
          'search_trends_hyperpigmentation',
          'search_trends_hypersomnia',
          'search_trends_hypertension',
          'search_trends_hyperthermia',
          'search_trends_hyperthyroidism',
          'search_trends_hypertriglyceridemia',
          'search_trends_hypertrophy',
          'search_trends_hyperventilation',
          'search_trends_hypocalcaemia',
          'search_trends_hypochondriasis',
          'search_trends_hypoglycemia',
          'search_trends_hypogonadism',
          'search_trends_hypokalemia',
          'search_trends_hypomania',
          'search_trends_hyponatremia',
          'search_trends_hypotension',
          'search_trends_hypothyroidism',
          'search_trends_hypoxemia',
          'search_trends_hypoxia',
          'search_trends_impetigo',
          'search_trends_implantation_bleeding',
          'search_trends_impulsivity',
          'search_trends_indigestion',
          'search_trends_infection',
          'search_trends_inflammation',
          'search_trends_inflammatory_bowel_disease',
          'search_trends_ingrown_hair',
          'search_trends_insomnia',
          'search_trends_insulin_resistance',
          'search_trends_intermenstrual_bleeding',
          'search_trends_intracranial_pressure',
          'search_trends_iron_deficiency',
          'search_trends_irregular_menstruation',
          'search_trends_itch',
          'search_trends_jaundice',
          'search_trends_kidney_failure',
          'search_trends_kidney_stone',
          'search_trends_knee_pain',
          'search_trends_kyphosis',
          'search_trends_lactose_intolerance',
          'search_trends_laryngitis',
          'search_trends_leg_cramps',
          'search_trends_lesion',
          'search_trends_leukorrhea',
          'search_trends_lightheadedness',
          'search_trends_low_back_pain',
          'search_trends_low_grade_fever',
          'search_trends_lymphedema',
          'search_trends_major_depressive_disorder',
          'search_trends_malabsorption',
          'search_trends_male_infertility',
          'search_trends_manic_disorder',
          'search_trends_melasma',
          'search_trends_melena',
          'search_trends_meningitis',
          'search_trends_menorrhagia',
          'search_trends_middle_back_pain',
          'search_trends_migraine',
          'search_trends_milium',
          'search_trends_mitral_insufficiency',
          'search_trends_mood_disorder',
          'search_trends_mood_swing',
          'search_trends_morning_sickness',
          'search_trends_motion_sickness',
          'search_trends_mouth_ulcer',
          'search_trends_muscle_atrophy',
          'search_trends_muscle_weakness',
          'search_trends_myalgia',
          'search_trends_mydriasis',
          'search_trends_myocardial_infarction',
          'search_trends_myoclonus',
          'search_trends_nasal_congestion',
          'search_trends_nasal_polyp',
          'search_trends_nausea',
          'search_trends_neck_mass',
          'search_trends_neck_pain',
          'search_trends_neonatal_jaundice',
          'search_trends_nerve_injury',
          'search_trends_neuralgia',
          'search_trends_neutropenia',
          'search_trends_night_sweats',
          'search_trends_night_terror',
          'search_trends_nocturnal_enuresis',
          'search_trends_nodule',
          'search_trends_nosebleed',
          'search_trends_nystagmus',
          'search_trends_obesity',
          'search_trends_onychorrhexis',
          'search_trends_oral_candidiasis',
          'search_trends_orthostatic_hypotension',
          'search_trends_osteopenia',
          'search_trends_osteophyte',
          'search_trends_osteoporosis',
          'search_trends_otitis',
          'search_trends_otitis_externa',
          'search_trends_otitis_media',
          'search_trends_pain',
          'search_trends_palpitations',
          'search_trends_pancreatitis',
          'search_trends_panic_attack',
          'search_trends_papule',
          'search_trends_paranoia',
          'search_trends_paresthesia',
          'search_trends_pelvic_inflammatory_disease',
          'search_trends_pericarditis',
          'search_trends_periodontal_disease',
          'search_trends_periorbital_puffiness',
          'search_trends_peripheral_neuropathy',
          'search_trends_perspiration',
          'search_trends_petechia',
          'search_trends_phlegm',
          'search_trends_photodermatitis',
          'search_trends_photophobia',
          'search_trends_photopsia',
          'search_trends_pleural_effusion',
          'search_trends_pleurisy',
          'search_trends_pneumonia',
          'search_trends_podalgia',
          'search_trends_polycythemia',
          'search_trends_polydipsia',
          'search_trends_polyneuropathy',
          'search_trends_polyuria',
          'search_trends_poor_posture',
          'search_trends_post_nasal_drip',
          'search_trends_postural_orthostatic_tachycardia_syndrome',
          'search_trends_prediabetes',
          'search_trends_proteinuria',
          'search_trends_pruritus_ani',
          'search_trends_psychosis',
          'search_trends_ptosis',
          'search_trends_pulmonary_edema',
          'search_trends_pulmonary_hypertension',
          'search_trends_purpura',
          'search_trends_pus',
          'search_trends_pyelonephritis',
          'search_trends_radiculopathy',
          'search_trends_rectal_pain',
          'search_trends_rectal_prolapse',
          'search_trends_red_eye',
          'search_trends_renal_colic',
          'search_trends_restless_legs_syndrome',
          'search_trends_rheum',
          'search_trends_rhinitis',
          'search_trends_rhinorrhea',
          'search_trends_rosacea',
          'search_trends_round_ligament_pain',
          'search_trends_rumination',
          'search_trends_scar',
          'search_trends_sciatica',
          'search_trends_scoliosis',
          'search_trends_seborrheic_dermatitis',
          'search_trends_self_harm',
          'search_trends_sensitivity_to_sound',
          'search_trends_sexual_dysfunction',
          'search_trends_shallow_breathing',
          'search_trends_sharp_pain',
          'search_trends_shivering',
          'search_trends_shortness_of_breath',
          'search_trends_shyness',
          'search_trends_sinusitis',
          'search_trends_skin_condition',
          'search_trends_skin_rash',
          'search_trends_skin_tag',
          'search_trends_skin_ulcer',
          'search_trends_sleep_apnea',
          'search_trends_sleep_deprivation',
          'search_trends_sleep_disorder',
          'search_trends_snoring',
          'search_trends_sore_throat',
          'search_trends_spasticity',
          'search_trends_splenomegaly',
          'search_trends_sputum',
          'search_trends_stomach_rumble',
          'search_trends_strabismus',
          'search_trends_stretch_marks',
          'search_trends_stridor',
          'search_trends_stroke',
          'search_trends_stuttering',
          'search_trends_subdural_hematoma',
          'search_trends_suicidal_ideation',
          'search_trends_swelling',
          'search_trends_swollen_feet',
          'search_trends_swollen_lymph_nodes',
          'search_trends_syncope',
          'search_trends_tachycardia',
          'search_trends_tachypnea',
          'search_trends_telangiectasia',
          'search_trends_tenderness',
          'search_trends_testicular_pain',
          'search_trends_throat_irritation',
          'search_trends_thrombocytopenia',
          'search_trends_thyroid_nodule',
          'search_trends_tic',
          'search_trends_tinnitus',
          'search_trends_tonsillitis',
          'search_trends_toothache',
          'search_trends_tremor',
          'search_trends_trichoptilosis',
          'search_trends_tumor',
          'search_trends_type_2_diabetes',
          'search_trends_unconsciousness',
          'search_trends_underweight',
          'search_trends_upper_respiratory_tract_infection',
          'search_trends_urethritis',
          'search_trends_urinary_incontinence',
          'search_trends_urinary_tract_infection',
          'search_trends_urinary_urgency',
          'search_trends_uterine_contraction',
          'search_trends_vaginal_bleeding',
          'search_trends_vaginal_discharge',
          'search_trends_vaginitis',
          'search_trends_varicose_veins',
          'search_trends_vasculitis',
          'search_trends_ventricular_fibrillation',
          'search_trends_ventricular_tachycardia',
          'search_trends_vertigo',
          'search_trends_viral_pneumonia',
          'search_trends_visual_acuity',
          'search_trends_vomiting',
          'search_trends_wart',
          'search_trends_water_retention',
          'search_trends_weakness',
          'search_trends_weight_gain',
          'search_trends_wheeze',
          'search_trends_xeroderma',
          'search_trends_xerostomia',
          'search_trends_yawn',
          'new_persons_vaccinated',
          'cumulative_persons_vaccinated',
          'stringency_index',
          'average_temperature_celsius',
          'minimum_temperature_celsius',
          'maximum_temperature_celsius',
          'rainfall_mm',
          'snowfall_mm',
          'dew_point',
          'relative_humidity']

In [42]:
data

,location_key,date,population,population_density,new_recovered,new_deceased,new_confirmed,latitude,longitude,smoking_prevalence,...,new_persons_vaccinated,cumulative_persons_vaccinated,stringency_index,average_temperature_celsius,minimum_temperature_celsius,maximum_temperature_celsius,rainfall_mm,snowfall_mm,dew_point,relative_humidity
0,US_AK_02013,2020-01-01,3455.0,NaN,NaN,NaN,NaN,55.23,-161.92,NaN,...,NaN,NaN,NaN,-8.583333,-9.590278,-6.819444,0.41275,NaN,-12.077778,78.113561
1,US_AK_02013,2020-01-02,3455.0,NaN,NaN,NaN,NaN,55.23,-161.92,NaN,...,NaN,NaN,NaN,-8.854167,-10.770833,-7.701389,0.50800,NaN,-12.811111,75.823558
2,US_AK_02013,2020-01-03,3455.0,NaN,NaN,NaN,NaN,55.23,-161.92,NaN,...,NaN,NaN,NaN,-10.368056,-12.486111,-6.770833,0.44450,NaN,-14.577778,75.132879
3,US_AK_02013,2020-01-04,3455.0,NaN,NaN,NaN,NaN,55.23,-161.92,NaN,...,NaN,NaN,NaN,-6.944444,-9.993056,-3.937500,0.28575,NaN,-11.711111,73.476473
4,US_AK_02013,2020-01-05,3455.0,NaN,NaN,NaN,NaN,55.23,-161.92,NaN,...,NaN,NaN,NaN,-3.791667,-8.291667,1.013889,0.03175,NaN,-7.177778,82.382291
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3195970,US_WY_56045,2022-09-13,6968.0,NaN,NaN,NaN,NaN,43.84,-104.56,NaN,...,NaN,NaN,NaN,17.683333,6.338889,28.733333,0.00000,NaN,4.055556,39.426422
3195971,US_WY_56045,2022-09-14,6968.0,NaN,NaN,NaN,NaN,43.84,-104.56,NaN,...,NaN,NaN,NaN,20.333333,17.494444,23.766667,0.00000,NaN,4.562500,36.975309
3195972,US_WY_56045,2022-09-15,6968.0,NaN,NaN,NaN,NaN,43.84,-104.56,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3195973,US_WY_56045,2022-09-16,6968.0,NaN,NaN,NaN,NaN,43.84,-104.56,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [30]:
data = data.reset_index(drop=True)

In [57]:
# copy the data
data_copy = data.copy()

In [58]:
data.dropna(axis=1, how='all', inplace=True)

In [64]:
for column in data.columns:
    print(f"'{column}',")

'location_key',
'date',
'population',
'new_deceased',
'new_confirmed',
'latitude',
'longitude',
'life_expectancy',
'mobility_retail_and_recreation',
'mobility_grocery_and_pharmacy',
'mobility_parks',
'mobility_transit_stations',
'mobility_workplaces',
'mobility_residential',
'search_trends_abdominal_obesity',
'search_trends_abdominal_pain',
'search_trends_acne',
'search_trends_actinic_keratosis',
'search_trends_acute_bronchitis',
'search_trends_adrenal_crisis',
'search_trends_ageusia',
'search_trends_alcoholism',
'search_trends_allergic_conjunctivitis',
'search_trends_allergy',
'search_trends_amblyopia',
'search_trends_amenorrhea',
'search_trends_amnesia',
'search_trends_anal_fissure',
'search_trends_anaphylaxis',
'search_trends_anemia',
'search_trends_angina_pectoris',
'search_trends_angioedema',
'search_trends_angular_cheilitis',
'search_trends_anosmia',
'search_trends_anxiety',
'search_trends_aphasia',
'search_trends_aphonia',
'search_trends_apnea',
'search_trends_arthralgia',
'sear

In [66]:
features = ['population',
            'new_deceased',
            'new_confirmed',
            'latitude',
            'longitude',
            'life_expectancy',
            'mobility_retail_and_recreation',
            'mobility_grocery_and_pharmacy',
            'mobility_parks',
            'mobility_transit_stations',
            'mobility_workplaces',
            'mobility_residential',
            'search_trends_abdominal_obesity',
            'search_trends_abdominal_pain',
            'search_trends_acne',
            'search_trends_actinic_keratosis',
            'search_trends_acute_bronchitis',
            'search_trends_adrenal_crisis',
            'search_trends_ageusia',
            'search_trends_alcoholism',
            'search_trends_allergic_conjunctivitis',
            'search_trends_allergy',
            'search_trends_amblyopia',
            'search_trends_amenorrhea',
            'search_trends_amnesia',
            'search_trends_anal_fissure',
            'search_trends_anaphylaxis',
            'search_trends_anemia',
            'search_trends_angina_pectoris',
            'search_trends_angioedema',
            'search_trends_angular_cheilitis',
            'search_trends_anosmia',
            'search_trends_anxiety',
            'search_trends_aphasia',
            'search_trends_aphonia',
            'search_trends_apnea',
            'search_trends_arthralgia',
            'search_trends_arthritis',
            'search_trends_ascites',
            'search_trends_asperger_syndrome',
            'search_trends_asphyxia',
            'search_trends_asthma',
            'search_trends_astigmatism',
            'search_trends_ataxia',
            'search_trends_atheroma',
            'search_trends_attention_deficit_hyperactivity_disorder',
            'search_trends_auditory_hallucination',
            'search_trends_autoimmune_disease',
            'search_trends_avoidant_personality_disorder',
            'search_trends_back_pain',
            'search_trends_bacterial_vaginosis',
            'search_trends_balance_disorder',
            'search_trends_beaus_lines',
            'search_trends_bells_palsy',
            'search_trends_biliary_colic',
            'search_trends_binge_eating',
            'search_trends_bleeding',
            'search_trends_bleeding_on_probing',
            'search_trends_blepharospasm',
            'search_trends_bloating',
            'search_trends_blood_in_stool',
            'search_trends_blurred_vision',
            'search_trends_blushing',
            'search_trends_boil',
            'search_trends_bone_fracture',
            'search_trends_bone_tumor',
            'search_trends_bowel_obstruction',
            'search_trends_bradycardia',
            'search_trends_braxton_hicks_contractions',
            'search_trends_breakthrough_bleeding',
            'search_trends_breast_pain',
            'search_trends_bronchitis',
            'search_trends_bruise',
            'search_trends_bruxism',
            'search_trends_bunion',
            'search_trends_burn',
            'search_trends_burning_chest_pain',
            'search_trends_burning_mouth_syndrome',
            'search_trends_candidiasis',
            'search_trends_canker_sore',
            'search_trends_cardiac_arrest',
            'search_trends_carpal_tunnel_syndrome',
            'search_trends_cataplexy',
            'search_trends_cataract',
            'search_trends_chancre',
            'search_trends_cheilitis',
            'search_trends_chest_pain',
            'search_trends_chills',
            'search_trends_chorea',
            'search_trends_chronic_pain',
            'search_trends_cirrhosis',
            'search_trends_cleft_lip_and_cleft_palate',
            'search_trends_clouding_of_consciousness',
            'search_trends_cluster_headache',
            'search_trends_colitis',
            'search_trends_coma',
            'search_trends_common_cold',
            'search_trends_compulsive_behavior',
            'search_trends_compulsive_hoarding',
            'search_trends_confusion',
            'search_trends_congenital_heart_defect',
            'search_trends_conjunctivitis',
            'search_trends_constipation',
            'search_trends_convulsion',
            'search_trends_cough',
            'search_trends_crackles',
            'search_trends_cramp',
            'search_trends_crepitus',
            'search_trends_croup',
            'search_trends_cyanosis',
            'search_trends_dandruff',
            'search_trends_delayed_onset_muscle_soreness',
            'search_trends_dementia',
            'search_trends_dentin_hypersensitivity',
            'search_trends_depersonalization',
            'search_trends_depression',
            'search_trends_dermatitis',
            'search_trends_desquamation',
            'search_trends_developmental_disability',
            'search_trends_diabetes',
            'search_trends_diabetic_ketoacidosis',
            'search_trends_diarrhea',
            'search_trends_dizziness',
            'search_trends_dry_eye_syndrome',
            'search_trends_dysautonomia',
            'search_trends_dysgeusia',
            'search_trends_dysmenorrhea',
            'search_trends_dyspareunia',
            'search_trends_dysphagia',
            'search_trends_dysphoria',
            'search_trends_dystonia',
            'search_trends_dysuria',
            'search_trends_ear_pain',
            'search_trends_eczema',
            'search_trends_edema',
            'search_trends_encephalitis',
            'search_trends_encephalopathy',
            'search_trends_epidermoid_cyst',
            'search_trends_epilepsy',
            'search_trends_epiphora',
            'search_trends_erectile_dysfunction',
            'search_trends_erythema',
            'search_trends_erythema_chronicum_migrans',
            'search_trends_esophagitis',
            'search_trends_excessive_daytime_sleepiness',
            'search_trends_eye_pain',
            'search_trends_eye_strain',
            'search_trends_facial_nerve_paralysis',
            'search_trends_facial_swelling',
            'search_trends_fasciculation',
            'search_trends_fatigue',
            'search_trends_fatty_liver_disease',
            'search_trends_fecal_incontinence',
            'search_trends_fever',
            'search_trends_fibrillation',
            'search_trends_fibrocystic_breast_changes',
            'search_trends_fibromyalgia',
            'search_trends_flatulence',
            'search_trends_floater',
            'search_trends_focal_seizure',
            'search_trends_folate_deficiency',
            'search_trends_food_craving',
            'search_trends_food_intolerance',
            'search_trends_frequent_urination',
            'search_trends_gastroesophageal_reflux_disease',
            'search_trends_gastroparesis',
            'search_trends_generalized_anxiety_disorder',
            'search_trends_genital_wart',
            'search_trends_gingival_recession',
            'search_trends_gingivitis',
            'search_trends_globus_pharyngis',
            'search_trends_goitre',
            'search_trends_gout',
            'search_trends_grandiosity',
            'search_trends_granuloma',
            'search_trends_guilt',
            'search_trends_hair_loss',
            'search_trends_halitosis',
            'search_trends_hay_fever',
            'search_trends_headache',
            'search_trends_heart_arrhythmia',
            'search_trends_heart_murmur',
            'search_trends_heartburn',
            'search_trends_hematochezia',
            'search_trends_hematoma',
            'search_trends_hematuria',
            'search_trends_hemolysis',
            'search_trends_hemoptysis',
            'search_trends_hemorrhoids',
            'search_trends_hepatic_encephalopathy',
            'search_trends_hepatitis',
            'search_trends_hepatotoxicity',
            'search_trends_hiccup',
            'search_trends_hip_pain',
            'search_trends_hives',
            'search_trends_hot_flash',
            'search_trends_hydrocephalus',
            'search_trends_hypercalcaemia',
            'search_trends_hypercapnia',
            'search_trends_hypercholesterolemia',
            'search_trends_hyperemesis_gravidarum',
            'search_trends_hyperglycemia',
            'search_trends_hyperhidrosis',
            'search_trends_hyperkalemia',
            'search_trends_hyperlipidemia',
            'search_trends_hypermobility',
            'search_trends_hyperpigmentation',
            'search_trends_hypersomnia',
            'search_trends_hypertension',
            'search_trends_hyperthermia',
            'search_trends_hyperthyroidism',
            'search_trends_hypertriglyceridemia',
            'search_trends_hypertrophy',
            'search_trends_hyperventilation',
            'search_trends_hypocalcaemia',
            'search_trends_hypochondriasis',
            'search_trends_hypoglycemia',
            'search_trends_hypogonadism',
            'search_trends_hypokalemia',
            'search_trends_hypomania',
            'search_trends_hyponatremia',
            'search_trends_hypotension',
            'search_trends_hypothyroidism',
            'search_trends_hypoxemia',
            'search_trends_hypoxia',
            'search_trends_impetigo',
            'search_trends_implantation_bleeding',
            'search_trends_impulsivity',
            'search_trends_indigestion',
            'search_trends_infection',
            'search_trends_inflammation',
            'search_trends_inflammatory_bowel_disease',
            'search_trends_ingrown_hair',
            'search_trends_insomnia',
            'search_trends_insulin_resistance',
            'search_trends_intermenstrual_bleeding',
            'search_trends_intracranial_pressure',
            'search_trends_iron_deficiency',
            'search_trends_irregular_menstruation',
            'search_trends_itch',
            'search_trends_jaundice',
            'search_trends_kidney_failure',
            'search_trends_kidney_stone',
            'search_trends_knee_pain',
            'search_trends_kyphosis',
            'search_trends_lactose_intolerance',
            'search_trends_laryngitis',
            'search_trends_leg_cramps',
            'search_trends_lesion',
            'search_trends_leukorrhea',
            'search_trends_lightheadedness',
            'search_trends_low_back_pain',
            'search_trends_low_grade_fever',
            'search_trends_lymphedema',
            'search_trends_major_depressive_disorder',
            'search_trends_malabsorption',
            'search_trends_male_infertility',
            'search_trends_manic_disorder',
            'search_trends_melasma',
            'search_trends_melena',
            'search_trends_meningitis',
            'search_trends_menorrhagia',
            'search_trends_middle_back_pain',
            'search_trends_migraine',
            'search_trends_milium',
            'search_trends_mitral_insufficiency',
            'search_trends_mood_disorder',
            'search_trends_mood_swing',
            'search_trends_morning_sickness',
            'search_trends_motion_sickness',
            'search_trends_mouth_ulcer',
            'search_trends_muscle_atrophy',
            'search_trends_muscle_weakness',
            'search_trends_myalgia',
            'search_trends_mydriasis',
            'search_trends_myocardial_infarction',
            'search_trends_myoclonus',
            'search_trends_nasal_congestion',
            'search_trends_nasal_polyp',
            'search_trends_nausea',
            'search_trends_neck_mass',
            'search_trends_neck_pain',
            'search_trends_neonatal_jaundice',
            'search_trends_nerve_injury',
            'search_trends_neuralgia',
            'search_trends_neutropenia',
            'search_trends_night_sweats',
            'search_trends_night_terror',
            'search_trends_nocturnal_enuresis',
            'search_trends_nodule',
            'search_trends_nosebleed',
            'search_trends_nystagmus',
            'search_trends_obesity',
            'search_trends_onychorrhexis',
            'search_trends_oral_candidiasis',
            'search_trends_orthostatic_hypotension',
            'search_trends_osteopenia',
            'search_trends_osteophyte',
            'search_trends_osteoporosis',
            'search_trends_otitis',
            'search_trends_otitis_externa',
            'search_trends_otitis_media',
            'search_trends_pain',
            'search_trends_palpitations',
            'search_trends_pancreatitis',
            'search_trends_panic_attack',
            'search_trends_papule',
            'search_trends_paranoia',
            'search_trends_paresthesia',
            'search_trends_pelvic_inflammatory_disease',
            'search_trends_pericarditis',
            'search_trends_periodontal_disease',
            'search_trends_periorbital_puffiness',
            'search_trends_peripheral_neuropathy',
            'search_trends_perspiration',
            'search_trends_petechia',
            'search_trends_phlegm',
            'search_trends_photodermatitis',
            'search_trends_photophobia',
            'search_trends_photopsia',
            'search_trends_pleural_effusion',
            'search_trends_pleurisy',
            'search_trends_pneumonia',
            'search_trends_podalgia',
            'search_trends_polycythemia',
            'search_trends_polydipsia',
            'search_trends_polyneuropathy',
            'search_trends_polyuria',
            'search_trends_poor_posture',
            'search_trends_post_nasal_drip',
            'search_trends_postural_orthostatic_tachycardia_syndrome',
            'search_trends_prediabetes',
            'search_trends_proteinuria',
            'search_trends_pruritus_ani',
            'search_trends_psychosis',
            'search_trends_ptosis',
            'search_trends_pulmonary_edema',
            'search_trends_pulmonary_hypertension',
            'search_trends_purpura',
            'search_trends_pus',
            'search_trends_pyelonephritis',
            'search_trends_radiculopathy',
            'search_trends_rectal_pain',
            'search_trends_rectal_prolapse',
            'search_trends_red_eye',
            'search_trends_renal_colic',
            'search_trends_restless_legs_syndrome',
            'search_trends_rheum',
            'search_trends_rhinitis',
            'search_trends_rhinorrhea',
            'search_trends_rosacea',
            'search_trends_round_ligament_pain',
            'search_trends_rumination',
            'search_trends_scar',
            'search_trends_sciatica',
            'search_trends_scoliosis',
            'search_trends_seborrheic_dermatitis',
            'search_trends_self_harm',
            'search_trends_sensitivity_to_sound',
            'search_trends_sexual_dysfunction',
            'search_trends_shallow_breathing',
            'search_trends_sharp_pain',
            'search_trends_shivering',
            'search_trends_shortness_of_breath',
            'search_trends_shyness',
            'search_trends_sinusitis',
            'search_trends_skin_condition',
            'search_trends_skin_rash',
            'search_trends_skin_tag',
            'search_trends_skin_ulcer',
            'search_trends_sleep_apnea',
            'search_trends_sleep_deprivation',
            'search_trends_sleep_disorder',
            'search_trends_snoring',
            'search_trends_sore_throat',
            'search_trends_spasticity',
            'search_trends_splenomegaly',
            'search_trends_sputum',
            'search_trends_stomach_rumble',
            'search_trends_strabismus',
            'search_trends_stretch_marks',
            'search_trends_stridor',
            'search_trends_stroke',
            'search_trends_stuttering',
            'search_trends_subdural_hematoma',
            'search_trends_suicidal_ideation',
            'search_trends_swelling',
            'search_trends_swollen_feet',
            'search_trends_swollen_lymph_nodes',
            'search_trends_syncope',
            'search_trends_tachycardia',
            'search_trends_tachypnea',
            'search_trends_telangiectasia',
            'search_trends_tenderness',
            'search_trends_testicular_pain',
            'search_trends_throat_irritation',
            'search_trends_thrombocytopenia',
            'search_trends_thyroid_nodule',
            'search_trends_tic',
            'search_trends_tinnitus',
            'search_trends_tonsillitis',
            'search_trends_toothache',
            'search_trends_tremor',
            'search_trends_trichoptilosis',
            'search_trends_tumor',
            'search_trends_type_2_diabetes',
            'search_trends_unconsciousness',
            'search_trends_underweight',
            'search_trends_upper_respiratory_tract_infection',
            'search_trends_urethritis',
            'search_trends_urinary_incontinence',
            'search_trends_urinary_tract_infection',
            'search_trends_urinary_urgency',
            'search_trends_uterine_contraction',
            'search_trends_vaginal_bleeding',
            'search_trends_vaginal_discharge',
            'search_trends_vaginitis',
            'search_trends_varicose_veins',
            'search_trends_vasculitis',
            'search_trends_ventricular_fibrillation',
            'search_trends_ventricular_tachycardia',
            'search_trends_vertigo',
            'search_trends_viral_pneumonia',
            'search_trends_visual_acuity',
            'search_trends_vomiting',
            'search_trends_wart',
            'search_trends_water_retention',
            'search_trends_weakness',
            'search_trends_weight_gain',
            'search_trends_wheeze',
            'search_trends_xeroderma',
            'search_trends_xerostomia',
            'search_trends_yawn',
            'average_temperature_celsius',
            'minimum_temperature_celsius',
            'maximum_temperature_celsius',
            'rainfall_mm',
            'snowfall_mm',
            'dew_point',
            'relative_humidity']

In [67]:
data_array = np.zeros((date.shape[0], location_key.shape[0], len(features)))
data_array.shape
for i in range(date.shape[0]):
    d = np.where(date[i] == data['date'])
    tmp = data.iloc[d]
    for j in range(location_key.shape[0]):
        l = np.where(location_key[j] == tmp['location_key'])
        data_array[i, j, :] = tmp.iloc[l][features].values
    

In [73]:
data_array_small = data_array.astype(np.float16)

In [74]:
data_array_small[0,0,:]

array([ 3.456e+03,        nan,        nan,  5.522e+01, -1.619e+02,
        8.688e+01,        nan,        nan,        nan,        nan,
              nan,        nan,        nan,        nan,        nan,
              nan,        nan,        nan,        nan,        nan,
              nan,        nan,        nan,        nan,        nan,
              nan,        nan,        nan,        nan,        nan,
              nan,        nan,        nan,        nan,        nan,
              nan,        nan,        nan,        nan,        nan,
              nan,        nan,        nan,        nan,        nan,
              nan,        nan,        nan,        nan,        nan,
              nan,        nan,        nan,        nan,        nan,
              nan,        nan,        nan,        nan,        nan,
              nan,        nan,        nan,        nan,        nan,
              nan,        nan,        nan,        nan,        nan,
              nan,        nan,        nan,        nan,        

In [76]:
np.save('data_array.npy', data_array)

In [78]:
position = np.where(date == '2021-04-15')
print(position)

(array([470]),)


In [82]:
timeframe = 14
dates = date[0:timeframe+1]
print('Start at: ', dates[0])
print('End at: ', dates[-1])

Start at:  2020-01-01
End at:  2020-01-15


In [ ]:

def GraphGenerator(data):
    # for a single instant in time, build a graph with nodes as counties and edges as mobility between counties

    ''' 
    Daily Flow Data 
    There are 594160 links of which across each link the following is noted
    geoid_o - Unique identifier of the origin geographic unit (census tract, county, and state). Type: string.
    geoid_d - Unique identifier of the destination geographic unit (census tract, county, and state). Type: string.
    lat_o - Latitude of the geometric centroid of the origin unit. Type: float.
    lng_o - Longitude of the geometric centroid of the origin unit. Type: float.
    lat_d - Latitude of the geometric centroid of the destination unit. Type: float.
    lng_d - Longitude of the geometric centroid of the destination unit. Type: float.
    date - Date of the records. Type: string.
    visitor_flows - Estimated number of visitors between the two geographic units (from geoid_o to geoid_d). Type: float.
    pop_flows - Estimated population flows between the two geographic units (from geoid_o to geoid_d), inferred from visitor_flows. Type: float.
    '''
    def normalize(column):
        weights = data[column]
        max_weight = np.max(weights)
        min_weight = np.min(weights)
        weight_range = max_weight - min_weight
        normalized_edge_weights = (weights - min_weight) / weight_range

        # Set the values directly and return them
        data[column] = normalized_edge_weights
        return normalized_edge_weights

    def skew_towards_maximum(column, skew=10):
        weights = data[column]
        max_weight = np.max(weights)
        min_weight = np.min(weights)
        weight_range = max_weight - min_weight

        # Perform the skew
        shifted = weights - min_weight + 1  # Shift values to avoid log(0)
        log_values = np.log(shifted) / np.log(skew)

        # Scale the log-transformed values back to the original range
        skewed_weights = (log_values / np.max(log_values)) * weight_range + min_weight

        # Set the values directly and return them
        data[column] = skewed_weights
        return skewed_weights

    # initialize plot
    fig, ax = plt.subplots()

    # index by geoid of both target and origin, and merge them to we ensure we have a list of all nodes
    geoid_indexed_data = data[['geoid_o', 'lat_o', 'lng_o']].drop_duplicates(subset=['geoid_o']).set_index('geoid_o')
    geoid_d_indexed_data = data[['geoid_d', 'lat_d', 'lng_d']].drop_duplicates(subset=['geoid_d']).set_index('geoid_d').rename(index={'geoid_d': 'geoid_o'}).rename(index={'lat_d': 'lat_o'}).rename(index={'lng_d': 'lng_o'})

    geoid_indexed_data.update(geoid_d_indexed_data)

    # plot the nodes
    lngs = geoid_indexed_data['lng_o']
    lats = geoid_indexed_data['lat_o']
    ax.scatter(lngs, lats, color='red', s=0.1)

    # filter out selfloops
    data = data.query('geoid_o != geoid_d')

    #normalize population flows
    normalize('pop_flows')

    #normalize visitor flows
    weights = skew_towards_maximum('visitor_flows', skew=10)
    print(weights[:10])
    weights = normalize('visitor_flows')
    print(weights[:10])

    # only show data for visitor flows above some threshold
    #data = data.query('visitor_flows > 0.15')

    # keep k highest value edges for each from_node
    k=5
    data = data.sort_values('visitor_flows').groupby('geoid_o').head(k)

    #TODO:  keep edges such that the top 80% of population_flow per node is accounted for

    # plot the edges
    starts = [data['lng_o'], data['lat_o']]
    ends = [data['lng_d'], data['lat_d']]
    lines = np.array(list(zip(starts, ends))).T
    lines = LineCollection(lines, linewidths=.1*weights)
    ax.add_collection(lines)
    plt.show()

    # Get adjacency matrix for out graph
    #graph.remove_edges_from(nx.selfloop_edges(G_whole))
    graph = nx.from_pandas_edgelist(data, 'geoid_o', 'geoid_d', ['visitor_flows'])
    S = nx.adjacency_matrix(graph)

    # check if S is connected
    print(nx.is_connected(graph))

    def calculate_sparsity(graph):
        M = nx.to_scipy_sparse_array(graph)
        matrix_size = len(graph.nodes) ** 2
        sparsity = (matrix_size - M.nnz) / matrix_size
        return sparsity

    def draw_network(coo_matrix, node_size=10, width=0.1):
        graph = nx.from_numpy_array(coo_matrix.toarray())
        nx.draw_networkx(graph, \
            node_size=node_size, width=width, with_labels=False)
        print(f"Sparsity: {calculate_sparsity(graph)}")
        plt.show()

    # create timegraph
    timegraph_size=2
    St = np.zeros((timegraph_size,timegraph_size))
    for i in range(1, timegraph_size):
        St[i, i-1] = 1

    # Kronecker
    kronecker_product = sp.sparse.kron(St, S)
    # draw_network(kronecker_product)

    # Cartesian
    # From paper On Cartesian product of matrices by Deepak Sarma: Cartesian product of two square matrices Aand Bas A&B=A⊗J+J⊗B, where J is the all one matrix of appropriate order and ⊗is the Kronecker product
    cartesian_product = sp.sparse.kron(St, np.identity(S.shape[0])) + sp.sparse.kron(np.identity(St.shape[0]), S)
    # draw_network(cartesian_product)

    # Strong
    strong_product = kronecker_product + cartesian_product
    # draw_network(strong_product)

    return graph, S, kronecker_product, cartesian_product, strong_product




In [ ]:
url = 'https://github.com/GeoDS/COVID19USFlows-DailyFlows/tree/master/daily_flows/county2county/daily_county2county_'
for d in dates:
    d = d[:4] + '_' + d[5:7] + '_' + d[8:]
    url_full = url+f'{d}.csv'
    response = requests.get(url_full)
    flow = pd.read_csv(url_full)
    _, adj_matrix, _, _, _ = GraphGenerator(flow)


In [103]:
# Download the daily flow data for a specific date range
start_year = "2020"
start_month = "01"
start_day = "01"
end_year = "2020"
end_month = "01"
end_day = "15"
output_folder = "daily_flows"

# Construct the command
command = [
    "python", "download_daily_data.py",
    "--start_year", start_year,
    "--start_month", start_month,
    "--start_day", start_day,
    "--end_year", end_year,
    "--end_month", end_month,
    "--end_day", end_day,
    "--output_folder", output_folder,
    "--county"
]

# Run the command
result = subprocess.run(command, capture_output=True, text=True)

# Print the result(if needed)
print(result.stdout)

In [ ]:
# 10 features